In [18]:
import numpy as np
import operator
import sklearn
from sklearn import datasets
from sklearn import linear_model
from sklearn.model_selection import train_test_split
"""
Property A.1: Algorithm converges to the maximum price vector over time
Description of Experiment 1.1
Take n example datasets from Scikit-Learn [(X_i), Y_i]
[(X_1), (X_2), …, (X_n)] are features on sale
[Y_1, Y_2, …., Y_n] are prediction tasks “for hire”
[b_1, b_2, ….., b_n] are bids for each prediction type  
Dynamics
Uniformly at random select (Y_n, b_n) as stream of buyers
Desired outcomes of experiment  
(P_i) converge to approximately b_i 
Why? Optimal Outcome is that price of each (feature set type) converges to bid for that that prediction task type
Regularization helps
Why? - Theory shows that it makes the problem “convex”
"""
dset_names = ['load_boston',
 'load_breast_cancer',
 'load_diabetes',
 'load_digits',
 'load_iris',
 'load_linnerud',
 'load_wine']
    
dsets = {}
for attr in dset_names:
    name = attr.split('load_')[1]
    if name:
        dsets[name] = getattr(datasets, attr)()

In [19]:
Y = [dsets[name]['target'] for name in dsets]

In [20]:
X = [dsets[name]['data'] for name in dsets]

In [21]:
X[1].shape

(569, 30)

In [22]:
[(x.size, i) for i, x in enumerate(Y)]

[(506, 0), (569, 1), (442, 2), (1797, 3), (150, 4), (60, 5), (178, 6)]

In [23]:
""" Look at first four, up to 400 time steps"""
X_t1 = [x[:400]for x in X[:4]]
Y_t1 = [y[:400] for y in Y[:4]]

In [24]:
SEED = 101
# 70% train, 20% validate, 10% test
X_train, X_test, y_train, y_test = train_test_split(X_t1[0], Y_t1[0], test_size=0.1, random_state=SEED)

In [25]:
clf = linear_model.LassoCV(cv=5, random_state=SEED)
clf.fit(X_train, y_train)

LassoCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
    max_iter=1000, n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=101, selection='cyclic', tol=0.0001,
    verbose=False)

In [27]:
clf.score(X_test, y_test)

0.4414395287471896

In [28]:
P = []

In [33]:
# (Algorithm2) CF(bn,  Pn,  rn,  Yn,  n,  c,B,  ρn)
#  1: fbnn(Pn)  =−PF(Pn,  bn,  Yn)
#  2: for all i∈[M] do 
#  3:   if Pn(i)≤bn then
#  4:     Pˆn(i) = Pˆn
#  5:   else if bn<Pn(i)≤bn+1/c then
#  6:     Pˆn(i)=Pn+c2(Pn−bn)2
#  7:   else
#  8:     Pˆn(i)=bn+12c
#  9:   end if
# 10: end for
# 11: gn = ∂{X=Pn}  ̄fbnn,c(X)
# 12: P'n+1 = Pn − ηgn
# 13: return Pn+1

In [ ]:
# TODO: assert(X_n) is numpy array
def payment_func(P_n, b_n, Y_n, context):
    """
    P_n: the price vector at previous timestep (t=n)
    b_n: the bid at previous timestep (in $ / model_score)
    Y_n: the test Y data set
    """
    feature_options = sorted(list((idx, p) for (idx, p) in enumerate(P_n) if p <= b_n), key=operator.itemgetter(1))
    gain_func = context.get('gain_func')
    if gain_func is None:
        return None
    
    X_n = context.get('X_n')
    X_n_train, X_n_test, y_n_train, y_n_test = train_test_split(X_n, Y_n, test_size=0.1, random_state=SEED)
    
    # Gain results by slice index
    feature_indices = []
    gain_results = []
    for (idx, (feature_idx, feature_price)) in enumerate(feature_options):
        _train = get_original_columns(X_n_train, feature_idxs=feature_options[:idx+1])
        _test = get_original_columns(X_n_test, feature_idxs=feature_options[:idx+1])
        clf.fit(_train, y_n_train)
        gain_results[idx] = clf.score(_test, y_n_test) # TODO: change to RMSE?
        feature_indices.append(feature_idx)

    
    # TODO: refactor
    positive_amount = b_n * gain_results[-1]
    negative_amount = 0
    # Add the bid as the rightmost price
    feature_indices += [-1]
    P_n += [b_n]
    for idx, gain in enumerate(gain_results):
        price_idx = feature_indices[idx]
        price_idx_plus_one = feature_indices[idx+1]
        price_difference = P_n[price_idx_plus_one] - P_n[price_idx]
        assert(price_difference >= 0)
        negative_amount += price_difference * gain
    return positive_amount - negative_amount


def get_original_columns(X_n, feature_idxs):
    """
    https://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.ix_.html
    TODO: figure out how to use np.ix_
    """
    

def algorithm2():
    pass


context = {
    'gain_func': clf,
    'X_n': X_n,
}
payment_func(P_n, b_n, Y_n, context)